In [2]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Outline:

## We are going to scrap https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm
## we'll get a list of movie name's along with we'll get movie name,movie url
## format of each movie 
## Movie Name,Movie Released Year,Movie overview,Movie Cast

In [3]:
import requests

## Use requests library to download web pages


In [4]:
movies_url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'

In [5]:
response = requests.get(movies_url)

In [6]:
response.status_code

200

In [7]:
len(response.text)

334544

In [8]:
page_content = response.text

In [9]:
with open('webpage.html','w',encoding="utf-8") as f:
    f.write(page_content)

# Use Beautiful Soap to parse and extract information

In [10]:
pip install beautifulsoup4 --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [11]:
from bs4 import BeautifulSoup

# Here the Doc contains plain Html 

In [12]:
doc = BeautifulSoup(page_content,'html.parser')

# Tip To parse the content:
### Use Loops while finding for different Nested Elements

In [13]:
def get_movie_name(doc):
    movie_name = []
    for tag in doc.find_all('td',class_='titleColumn'):
        for atag in tag.find_all('a'):
            movie_name.append(atag.text)
            
    return movie_name        

In [14]:
titles = get_movie_name(doc)

In [15]:
print(type(titles))


<class 'list'>


In [16]:
def get_movie_url(doc):
    movie_url = []
    for tag in doc.find_all('td',class_='titleColumn'):
        for atag in tag.find_all('a'):
            base_url = 'https://www.imdb.com'
            movie_url.append(base_url+atag['href'])
            
    return movie_url 
    

In [17]:
movieURLs = get_movie_url(doc)
len(movieURLs)

100

In [18]:
def get_rating(doc):
    movie_rating = []
    k = 0
    for tag in doc.find_all('td',class_='ratingColumn imdbRating'):
        if tag.strong!=None:
            movie_rating.append(tag.strong.text)
        else:
            movie_rating.insert(k,0)
        k = k+1    
    return movie_rating
    

In [19]:
movie_ratings = get_rating(doc)
movie_ratings[3]


'7.6'

In [20]:
import pandas as pd

In [21]:
movies_list= {
    'title':titles,
    'rating':movie_ratings,
    'url':movieURLs
}

In [22]:
movies_list_df = pd.DataFrame(movies_list)

# Here is the File with Movie Title,Movie Rating,Movie URL


In [90]:
movies_list

In [24]:
movies_list_df

,title,rating,url
0,The Flash,7.3,https://www.imdb.com/title/tt0439572/
1,Spider-Man: Across the Spider-Verse,9.0,https://www.imdb.com/title/tt9362722/
2,Transformers: Rise of the Beasts,6.6,https://www.imdb.com/title/tt5090568/
3,Avatar: The Way of Water,7.6,https://www.imdb.com/title/tt1630029/
4,The Little Mermaid,7.2,https://www.imdb.com/title/tt5971474/
...,...,...,...
95,Harry Potter and the Sorcerer's Stone,7.6,https://www.imdb.com/title/tt0241527/
96,Joy Ride,6.5,https://www.imdb.com/title/tt15268244/
97,Bumblebee,6.7,https://www.imdb.com/title/tt4701182/
98,Indiana Jones and the Last Crusade,8.2,https://www.imdb.com/title/tt0097576/


In [25]:
movies_list_df.to_csv('movies_list.csv')

In [26]:
movies_list_df.url[0]

'https://www.imdb.com/title/tt0439572/'

# Here Access has been denied by server so we need a header to tell the server that we are not with Unauthorized access  

In [28]:
headers = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

In [29]:
response = requests.get(movies_list_df.url[0],headers= headers)

In [30]:
response.status_code

200

In [31]:
page_content = response.text

In [32]:
doc = BeautifulSoup(page_content,'html.parser')

# Tip : Try to implement For a Single Page At a time and then Apply Loop to iterate over pages

In [33]:
for tag in doc.find_all('div',class_='sc-52d569c6-0 kNzJA-D'):
    for spantag in tag.find_all('span',class_ = 'sc-afe43def-1 fDTGTb'):
        print(spantag.text)
        
    a = []
    for ultag in tag.find_all('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'):
        for litag in ultag.find_all('li',class_='ipc-inline-list__item'):
            a.append(litag.text)
    print(a[0])        
            
       
    for spantag in doc.find_all('span',class_= 'sc-cd57914c-2 cXtXlS'):
        print(spantag.text)

The Flash
2023
Barry Allen uses his super speed to change the past, but his attempt to save his family creates a world without super heroes, forcing him to race for his life in order to save the future.


# Here is the  Final Solution

In [75]:
top_100_most_popular_movies={
    'Movie Name':[],
    'Movie Released Year':[],
    'Movie overview':[],
    'Movie Cast':[]
}

In [74]:
def getcast(doc):
    arr = []
    for div1tag in doc.find_all('div',class_='ipc-sub-grid ipc-sub-grid--page-span-2 ipc-sub-grid--wraps-at-above-l ipc-shoveler__grid'):
        for div2 in div1tag.find_all('div',class_='sc-bfec09a1-5 kUzsHJ'):
            for div3 in div2.find_all('div',class_='sc-bfec09a1-7 dpBDvu'):
                arr.append(div3.a.text)
                

    return arr         

In [76]:
def getMovieName(doc):
    a = []
    for tag in doc.find_all('div',class_='sc-52d569c6-0 kNzJA-D'):
        for spantag in tag.find_all('span',class_ = 'sc-afe43def-1 fDTGTb'):
            a.append(spantag.text)
    return a[0]       

In [77]:
def getnameyearoverviewcast(doc):
    
    top_100_most_popular_movies['Movie Name'].append(getMovieName(doc))

    a = []
    for ultag in tag.find_all('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'):
        for litag in ultag.find_all('li',class_='ipc-inline-list__item'):
            a.append(litag.text)
    top_100_most_popular_movies['Movie Released Year'].append(a[0])
       
    for spantag in doc.find_all('span',class_= 'sc-cd57914c-2 cXtXlS'):
        top_100_most_popular_movies['Movie overview'].append(spantag.text)
    
    
    top_100_most_popular_movies['Movie Cast'].append(getcast(doc))
    
            
    return top_100_most_popular_movies

In [88]:
for i in range(100):
    response = requests.get(movies_list_df.url[i],headers= headers)
    page_content = response.text
    doc = BeautifulSoup(page_content,'html.parser')
    print('Currently Scrapping>{}'.format(movies_list_df['title'][i]))
    top = getnameyearoverviewcast(doc)
    

In [69]:
print(type(top_100_most_popular_movies['Movie Name']))

<class 'list'>


In [79]:
len(top_100_most_popular_movies['Movie Cast'])

100

In [80]:
len(top_100_most_popular_movies['Movie Released Year'])

100

In [81]:
len(top_100_most_popular_movies['Movie overview'])

100

In [87]:
top_100_most_popular_movies

In [83]:
a = pd.DataFrame(top_100_most_popular_movies)

In [84]:
a

,Movie Name,Movie Released Year,Movie overview,Movie Cast
0,The Flash,2023,Barry Allen uses his super speed to change the...,"[Ezra Miller, Michael Keaton, Sasha Calle, Mic..."
1,Spider-man: Across the Spider-verse,2023,"Miles Morales catapults across the Multiverse,...","[Shameik Moore, Hailee Steinfeld, Brian Tyree ..."
2,Transformers: Rise of the Beasts,2023,"During the '90s, a new faction of Transformers...","[Anthony Ramos, Dominique Fishback, Luna Laure..."
3,Avatar: The Way of Water,2023,Jake Sully lives with his newfound family form...,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
4,The Little Mermaid,2023,A young mermaid makes a deal with a sea witch ...,"[Halle Bailey, Jonah Hauer-King, Melissa McCar..."
...,...,...,...,...
95,Harry Potter and the Philosopher's Stone,2023,An orphaned boy enrolls in a school of wizardr...,"[Daniel Radcliffe, Rupert Grint, Richard Harri..."
96,Joy Ride,2023,Follows four Asian-American friends as they bo...,"[Stephanie Hsu, Ashley Park, David Denman, She..."
97,Bumblebee,2023,"On the run in the year 1987, Bumblebee finds r...","[Hailee Steinfeld, Jorge Lendeborg Jr., John C..."
98,Indiana Jones and the Last Crusade,2023,"In 1938, after his father goes missing while p...","[Harrison Ford, Sean Connery, Alison Doody, De..."


In [89]:
a['Movie Cast'][0]

In [92]:
a.to_csv('movies.csv')